# Play with simple stencil and numba

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import numpy as np

In [3]:
N = 51 #number of grid points
dt = 1.e-4 #time step
L = float(1) #size of grid
nsteps = 2000 #number of time steps
dx = L/(N-1) #grid spacing
nplot = 20 #number of timesteps before plotting

r = dt/dx**2 #assuming heat diffusion coefficient == 1

#initialize matrices A, B and b array
A = np.zeros((N-2,N-2))
B = np.zeros((N-2,N-2))
b = np.zeros((N-2))
#define matrices A, B and b array
for i in range(N-2):
    if i==0:
        A[i,:] = [2+2*r if j==0 else (-r) if j==1 else 0 for j in range(N-2)]
        B[i,:] = [2-2*r if j==0 else r if j==1 else 0 for j in range(N-2)]
        b[i] = 0. #boundary condition at i=1
    elif i==N-3:
        A[i,:] = [-r if j==N-4 else 2+2*r if j==N-3 else 0 for j in range(N-2)]
        B[i,:] = [r if j==N-4 else 2-2*r if j==N-3 else 0 for j in range(N-2)]
        b[i] = 0. #boundary condition at i=N
    else:
        A[i,:] = [-r if j==i-1 or j==i+1 else 2+2*r if j==i else 0 for j in range(N-2)]
        B[i,:] = [r if j==i-1 or j==i+1 else 2-2*r if j==i else 0 for j in range(N-2)]


In [4]:
x = np.linspace(0,1,N)
u = np.asarray([np.sin(xx*np.pi) for xx in x])
bb = B.dot(u[1:-1])

In [5]:
# %%timeit
u[1:-1] = np.linalg.solve(A,bb)

In [7]:
from numba import njit

In [8]:
def test(A,B,u):   
    for i in range(100):
        bb = B.dot(u[1:-1])
        u[1:-1] = np.linalg.solve(A,bb)
    return u

In [9]:
testjit = njit()(test)

In [10]:
_ = testjit(A,B,u)

In [15]:
%%timeit
nu = test(A,B,u)

3.72 ms ± 13.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [16]:
%%timeit
nu = testjit(A,B,u)

2.92 ms ± 4.41 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
